In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas  # noqa

hv.extension('bokeh')
BOKEH_OPTS = dict(height=600, width=1200)

# Mores Creek Summit
## 4 component Licor 200 Pyronometer

Created with:
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customMultiTimeSeriesGroupByStationReport/hourly/start_of_period/637:ID:SNTL%7Cid=%22%22%7Cname/2023-10-01,2024-04-19/LWINV::value,SWINV::value,LWOTV::value,SWOTV::value?fitToScreen=false

### Before 02/12

In [ ]:
radiation_data = pd.read_csv(
    '/data/Snowpack/ID-MCS-2024/NRCS-radiation-before-20240212.csv',
    header=0,
    comment='#',
    names=['Site', 'Date', 'Time', 'shortwave_in', 'shortwave_out', 'longwave_in', 'longwave_out', 'drop'],
).drop('Site', axis=1).drop('drop', axis=1)

radiation_data['datetime'] = pd.to_datetime(radiation_data.pop('Date') + 'T' + radiation_data.pop('Time') + '-08:00')
radiation_data

### After 02/12

In [ ]:
radiation_data_after = pd.read_csv(
    '/data/Snowpack/ID-MCS-2024/NRCS-radiation-after-20240212.csv',
    header=0,
    comment='#',
    names=['datetime', 'longwave_in', 'shortwave_in', 'longwave_out', 'shortwave_out'],
    dtype={'datetime': str, 'ISW': np.float64 ,'OSW': np.float64, 'ILW': np.float64, 'OLW': np.float64},
)
radiation_data_after['datetime'] = radiation_data_after['datetime'].astype(str) + ':00-08:00'
radiation_data_after['datetime'] = pd.to_datetime(radiation_data_after['datetime'], format='%Y-%m-%d %H:%M:%S%z')
radiation_data_after

### Merge

In [ ]:
radiation_data = pd.concat([radiation_data, radiation_data_after])

In [ ]:
# Data cleaning
radiation_data = radiation_data[radiation_data['datetime'] > pd.to_datetime('2023-10-11T00:00:00-08:00')]

radiation_data

In [ ]:
radiation_data.hvplot('datetime', **BOKEH_OPTS)

### Check merge time

In [ ]:
radiation_data[radiation_data['datetime'] > pd.to_datetime('2024-02-12T09:00:00-08:00')]

### Store in DB

In [ ]:
from hrrr_db.db.connection import SQLALCHEMY_ENGINE
from hrrr_db.db.models import Site, DataSource, Radiation

In [ ]:
MCS_record = Site.load('Mores Creek Summit')

### Add required record metadata

In [ ]:
radiation_data['site_id'] = MCS_record.id
radiation_data['data_source'] = DataSource.NRCS.name

radiation_data.head()

In [ ]:
radiation_data[['site_id', 'datetime', 'shortwave_in', 'shortwave_out', 'longwave_in', 'longwave_out', 'data_source']].to_sql(
    name='radiation', dtype={"data_source": Radiation.data_source.type},
    con=SQLALCHEMY_ENGINE, method='multi', if_exists='append', index=False,
    
)

#.to_csv('/data/tmp/radiation.csv', index=False)